In [ ]:
# Species Composition Classification using Sentinel-1 & Sentinel-2

# STEP 1: Import libraries
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.plot import show
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt

# STEP 2: Load raster bands (example for local processing, adjust for your setup)
# Use rasterio to read Sentinel bands and stack them
sentinel_bands = ["s2_b02.tif", "s2_b03.tif", "s2_b04.tif", "s2_b08.tif"]
stack = []
for band in sentinel_bands:
    with rasterio.open(band) as src:
        stack.append(src.read(1))

# Convert to NumPy array stack (bands, rows, cols)
img_stack = np.stack(stack)

# STEP 3: Example texture feature from Sentinel-1 VV band
def compute_texture_features(image, distances=[1], angles=[0]):
    glcm = greycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    texture = {
        'contrast': greycoprops(glcm, 'contrast')[0, 0],
        'homogeneity': greycoprops(glcm, 'homogeneity')[0, 0],
        'energy': greycoprops(glcm, 'energy')[0, 0],
        'correlation': greycoprops(glcm, 'correlation')[0, 0]
    }
    return texture

# Assume texture calculated for small patch - for full image, you’d run in a window loop (see skimage docs)

# STEP 4: Load training data (as GeoDataFrame with 'class' column)
training_data = gpd.read_file("training_samples.geojson")

# Extract raster values at sample points
def extract_features_at_points(raster_paths, points_gdf):
    features = []
    for idx, row in points_gdf.iterrows():
        values = []
        for r_path in raster_paths:
            with rasterio.open(r_path) as src:
                row_val, col_val = src.index(row.geometry.x, row.geometry.y)
                value = src.read(1)[row_val, col_val]
                values.append(value)
        features.append(values)
    return np.array(features)

X = extract_features_at_points(sentinel_bands, training_data)
y = training_data['class']

# STEP 5: Train classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# STEP 6: Evaluate
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
